# Introduction to Manim

At its core, Manim is a Python library that implements the capabilities of [FFmpeg](https://ffmpeg.org/) and
[OpenGL](https://www.opengl.org/). Manim can be used to make incredible animations, as evident in my favourite 3Blue1Brown
video on Fourier Transforms ([link](https://www.youtube.com/watch?v=spUNpyF58BY&t=627s)]. The library is large and complex,
so we will only cover the basics, here.

First, we need to import the capabilities of Manim. We will also set the maximum display width for videos to improve
their appearance in this notebook.

In [ ]:
from manim import *

config.media_width = "75%"

You should now see a printed message with the installed version of the Manim library.

## The Scene class

The `Scene` class is the basis for all Manim animations. To create an animation, you must define a Python class that
inherits `Scene` and implement a method called `construct`. The `construct` method is where you tell Manim what to do,
but you will see that we can also write other methods that can be called within `construct`.

Let's start with a simple animation of some text. The first step is to define a `Text` object with the desired string.
The `self.play()` function animates the text by calling the `Write` animation, and `self.wait()` pauses it for a moment.

In [ ]:
%%manim -qm HelloWorld
class HelloWorld(Scene):
    def construct(self):
        text = Text("Hello, astronomy people!")
        self.play(Write(text))
        self.wait()

To create the video we use the `-pqm` command, where:

-`p` is the preview flag, telling Manim that we want to immediately view the result and
-`qm` sets the quality to medium (others being low, high and 4k),

We can do a lot of different things to this `Text` object. We can move it around, make it go away, or even transform it
into a different text string! This time, we will define a `MathTex` equation and learn how to move it around the screen.

In [ ]:
%%manim -qm EquationMagic
class EquationMagic(Scene):
    def construct(self):
        text = Text("Hello, astronomy people!")
        self.play(Write(text))
        self.wait()

        equation = MathTex(r'H^2=H_0^2\left(\frac{\Omega_m}{a^3}+\frac{\Omega_r}{a^4}'
                           r'+\frac{\Omega_k}{a^2}+\Omega_{\Lambda}\right)')

        self.play(ReplacementTransform(text, equation))
        self.wait()
        self.play()
        self.wait()

        self.play(equation.animate.shift(LEFT))
        self.play(equation.animate.shift(RIGHT))
        self.play(equation.animate.shift(UP))
        self.play(equation.animate.shift(DOWN))
        self.wait()

        self.play(equation.animate.shift(3*UP))
        self.play(equation.animate.shift(8*DOWN))
        self.play(equation.animate.shift(5*UP))
        self.play(equation.animate.shift(7*LEFT))
        self.play(equation.animate.shift(7*RIGHT))
        self.wait()

Notice that we told the equation to move around with the `LEFT`, `RIGHT`, `UP`, and `DOWN` directions. For the last part
of the animation we moved the equation in incremental steps. The defualt Manim scene is a 8 x 14 grid.

There are lots of fun things we can do with equations. Check out this example from the Manim documentation:

In [ ]:
%%manim -qm MovingFrameBox
class MovingFrameBox(Scene):
    def construct(self):
        text=MathTex("\\frac{d}{dx}f(x)g(x)=", "f(x)\\frac{d}{dx}g(x)","+", "g(x)\\frac{d}{dx}f(x)")

        self.play(Write(text))

        framebox1 = SurroundingRectangle(text[1], buff = .1)
        framebox2 = SurroundingRectangle(text[3], buff = .1)

        self.play(Create(framebox1))
        self.wait()
        self.play(ReplacementTransform(framebox1,framebox2))
        self.wait()

## Shapes, shapes, shapes!
There are a lot of different shapes that you can easily create in Manim, such as `Circle`, `Square`, and `Triangle`.
To see all of the shape options, see the documentation [here](https://azarzadavila-manim.readthedocs.io/en/latest/geometry.html).

We're going to speed things up here. Let's create two shapes, a circle and a square, and see how we can animate them
at the same time. The following scene will demonstrate some additional animation options: `set_fill`, `scale`, and `rotate`.
Animation options can be seen [here](https://docs.manim.community/en/stable/reference_index/animations.html).
At the end, we will see how you can remove an object from the scene, either with `self.remove()` or `self.play(Fadeout())`.

In [ ]:
%%manim -qm Shapes
class Shapes(Scene):
    def construct(self):
        square = Square(color=BLUE, fill_opacity=1)
        circle = Circle(color=RED)

        self.play(square.animate.shift(3*LEFT), Create(circle))
        self.wait()
        self.play(square.animate.set_fill(ORANGE), circle.animate.shift(3*RIGHT))
        self.wait()
        self.play(square.animate.scale(0.5), circle.animate.scale(1.5))
        self.wait()
        self.play(square.animate.scale(1.5), circle.animate.scale(0.5))
        self.remove(circle)
        self.play(square.animate.rotate(0.4))
        self.wait()
        self.play(FadeOut(square))

## Animating Functions
With Manim, you can animate any mathematical function you can think of! In the following example we will define a set of
axes with the `Axes` object and draw a parabola. After that, we will make two dots with the `Dot` object and move them along
the curve at different speeds.

In [ ]:
%%manim -qm FunctionPlot
class FunctionPlot(Scene):
    def construct(self):
        ax = Axes( x_range=[-5, 5, 1], y_range=[0, 5, 1], x_length=10, y_length=5,
                   axis_config={"include_numbers": True})

        def parabola(x):
            return 0.2 * x**2

        curve = ax.plot(parabola, color=RED)
        self.play(Create(ax))
        self.play(Write(curve))
        self.wait()

        dot_1 = Dot(color=BLUE).scale(2)
        dot_2 = Dot(color=YELLOW).scale(2)
        self.play(MoveAlongPath(dot_1, curve, run_time=2), MoveAlongPath(dot_2, curve, run_time=4))
        self.wait()

## A More Complex Example
The following scene is one that I created for a presentation about exoplanet transit timing. I was learning Manim at the
time, so it may not be the most efficient implementation, but it should give you an idea of what you can do with the program.
This will take a lot longer to render than the previous scenes, so be patient!

In [ ]:
class OrbitingPlanet(Scene):
    def make_circular_orbit(self):
        self.star = Dot(color=YELLOW).scale(5)
        self.planet = Dot(color=DARK_BROWN).scale(2)
        self.circular_orbit = Circle(radius=2, color=GREY_A, stroke_width=1)
        self.circular_orbit.move_to(self.star.get_center())
        self.planet.move_to(self.circular_orbit.point_at_angle(0 * DEGREES))
        self.dashed_circle = DashedVMobject(self.circular_orbit, num_dashes=40)
        self.orbit = VGroup(self.star, self.dashed_circle, self.planet)
    def make_sky_plane(self):
        self.star_face = Circle(radius=2, color=DARK_BROWN).move_to(0.5 * DOWN + 3 * RIGHT)
        self.star_face.set_fill(YELLOW, opacity=1.5)
        start_point = self.star_face.point_at_angle(180 * DEGREES)
        end_point = self.star_face.point_at_angle(0 * DEGREES)
        self.transit_path = Line(start_point + 0.6 * LEFT, end_point + 0.5 * RIGHT, color=BLACK)
        self.transit_line = Line(start_point, end_point, color=PURE_RED, stroke_width=5)
        self.planet_face = Circle(color=BLACK, radius=0.2).move_to(start_point + 0.2 * LEFT)
        self.planet_face.set_fill(color=BLACK, opacity=1.0)
        self.sky_plane = VGroup(self.transit_path, self.star_face,
                                self.transit_line, self.planet_face)
        self.sky_plane_all = VGroup(self.sky_plane, self.planet_face)
    def make_arcs(self):
        self.arc0 = Arc(start_angle=0.0, angle=255*DEGREES, radius=2, color=BLACK,
                arc_center=self.star.get_center())
        self.arc1 = Arc(start_angle=255*DEGREES, angle=30*DEGREES, radius=2, color=BLACK,
                arc_center=self.star.get_center())
        self.arc2A = Arc(start_angle=285*DEGREES, angle=150*DEGREES, radius=2, color=BLACK,
                arc_center=self.star.get_center())
        self.arc2B = Arc(start_angle=285*DEGREES, angle=330*DEGREES, radius=2, color=BLACK,
                arc_center=self.star.get_center())
        self.arc3 = Arc(start_angle=75*DEGREES, angle=30*DEGREES, radius=2, color=BLACK,
                arc_center=self.star.get_center())
        self.arc4 = Arc(start_angle=105*DEGREES, angle=150*DEGREES, radius=2, color=BLACK,
                arc_center=self.star.get_center())
        self.arcs = VGroup(self.arc0, self.arc1, self.arc2A, self.arc2B, self.arc3, self.arc4)
    def make_lightcurve(self):
        def curve(t, min, max):
            if t < min: return 0
            elif min <= t < 0: return -t ** 3 - max ** 3
            elif 0 <= t <= max: return t ** 3 - max ** 3
            elif t > max: return 0

        def noise(t, min, max):
            return curve(t, min, max) + np.random.uniform(-2, 2)

        self.axes = Axes(tips=False, x_range=[0, 5], y_range=[-10, 10],
                    axis_config={"stroke_color":GREY_C,"stroke_width":2,"include_ticks": False,})
        self.flux_line = Line(start=np.array([-5., 0., 0.]), end=np.array([5., 0., 0.]),
                              color=PURE_RED)
        self.curve1 = FunctionGraph(lambda t: curve(t, -2, 2), x_range=[-5, -2],
                                    color=PURE_RED, stroke_width=5)
        self.curve2 = FunctionGraph(lambda t: curve(t, -2, 2), x_range=[-2, 2.1],
                                    color=PURE_RED, stroke_width=5)
        self.curve3 = FunctionGraph(lambda t: curve(t, -2, 2), x_range=[2, 5],
                                    color=PURE_RED, stroke_width=5)
        self.noise_curve = FunctionGraph(lambda t: noise(t, -2, 2), x_range=[-5, 5],
                                    color=BLUE, stroke_width=2)
        self.lightcurve = VGroup(self.curve1, self.curve2, self.curve3,
                                 self.noise_curve, self.axes, self.flux_line)
        self.t0_line = DashedLine(start=np.array([3, -2.5, 0.]), end=np.array([3, 1, 0.]),
                            dash_length=0.1, color=PURE_RED)
        self.lightcurve.scale(0.25)
        self.lightcurve.stretch(0.75, dim=1)
        self.lightcurve.stretch(2, dim=0)
        self.lightcurve.move_to([3, -1, 0])
        # self.labels = self.axes.get_axis_labels(x_label=Tex("Time"), y_label=Tex("Flux"))
    def construct(self):
        self.make_circular_orbit()
        self.make_lightcurve()
        self.make_sky_plane()
        self.make_arcs()

        title_transit = Tex("The Exoplanet Transit").to_corner(LEFT+UP)
        title_topdown = Tex("Top-Down View").move_to(1.75*UP + 3*LEFT)
        title_obs = Tex("Observer View").move_to(1.75 * UP + 3 * RIGHT)

        self.circular_orbit.shift(1.1 * DOWN + 3 * LEFT)
        self.orbit.shift(1.1 * DOWN + 3 * LEFT)
        self.arcs.shift(1.1 * DOWN + 3 * LEFT)

        sky_plane = VGroup(self.transit_path, self.star_face, self.transit_line, self.planet_face)
        sky_plane.shift(0.6*DOWN)

        self.add(title_topdown, self.orbit)
        self.wait()
        self.play(Create(self.star_face), Write(title_obs))
        self.wait()

        rt = 2
        self.play(MoveAlongPath(self.planet, self.arc0, run_time=rt, rate_func=linear))
        self.wait(0.1)

        transit_square = Square(side_length=0.8, color=PURE_RED) \
                                    .move_to(self.circular_orbit.point_at_angle(270 * DEGREES))
        self.add(transit_square, self.transit_line, self.planet_face)
        self.wait(0.1)

        self.play(MoveAlongPath(self.planet_face, self.transit_path, run_time=rt, rate_func=linear),
                  MoveAlongPath(self.planet, self.arc1, run_time=rt, rate_func=linear))

        self.play(MoveAlongPath(self.planet, self.arc2B, run_time=rt, rate_func=linear))
        self.play(MoveAlongPath(self.planet_face, self.transit_path, run_time=rt, rate_func=linear),
                  MoveAlongPath(self.planet, self.arc1, run_time=rt, rate_func=linear))
        self.wait()

        # move star to the left
        self.remove(self.circular_orbit)
        self.play(
                  FadeOut(self.orbit), sky_plane.animate.move_to(np.array([-3.5,-0.4,0])),
                  FadeOut(title_topdown), FadeOut(title_obs), FadeOut(transit_square))

        # show lightcurve creation
        xlabel = Tex("Time").move_to(np.array([3, -1, 0])).scale(0.75)
        ylabel = Tex("Flux").move_to(np.array([-0.5, 0.5, 0])).scale(0.75)
        self.add(self.axes)
        self.play(Write(xlabel), Write(ylabel), self.sky_plane.animate.scale(0.8))
        self.wait(0.5)

        self.play(Create(self.curve1, rate_func=linear, run_time=1),
                  xlabel.animate.shift(2.0 * DOWN))
        self.play(Create(self.curve2, rate_func=linear, run_time=1.5),
                  MoveAlongPath(self.planet_face, self.transit_path,
                                run_time=1.5, rate_func=linear), Write(title_transit))
        self.play(Create(self.curve3, rate_func=linear, run_time=1))

        self.wait()
        self.play(Create(self.noise_curve))
        self.remove(self.curve1, self.curve2, self.curve3)
        self.wait(2)

        title_t0 = Tex("Transit Timing").to_corner(LEFT+UP)
        self.play(Create(self.t0_line))
        self.wait(2)
        new_lightcurve = VGroup(self.noise_curve, self.axes, self.t0_line)
        self.play(FadeOut(self.sky_plane), FadeOut(xlabel), FadeOut(ylabel),
                  new_lightcurve.animate.shift(3 * LEFT), Transform(title_transit, title_t0))
        self.wait()


Now you have learned the basics of Manim. There are infinite possibilities! For some ideas, check out the
[Manim example gallery](https://docs.manim.community/en/stable/examples.html).